## Тестовое М_Видео

<div style="border:solid green 2px; padding: 20px"> <h1 style="color:green; margin-bottom:20px">Описание задачи</h1>

**Это одна из реальных задач**  
К нам пришел бизнес и попросил понять сколько уникальных SKU у нас в цепочках поставок для интернет магазина и прислали файл на листе `Data` от бизнеса  
Файл очень похож на реальный, только городов намного больше  
Они попросили сделать им алгоритм, который будет выдавать данные, как на листе `Output`  

**Логика алгоритма:**  
Есть город X. В нем есть интернет магазин. В зависимости от того, какой товар закажут, он может поехать со склада, или из магазина  
Приоритет выбора объекта `резерва`, как прописано в `цепочке` слева направо.  
Пример алгоритма на листе `Пример`

Лист `Data` - это данные из БД со стоками (в реальной жизни, данные берутся из БД напрямую)

В результате необходимо предоставить файл в формате Jupiter Notebook, который формирует данные, как на листе `Output`

</div>

**Выполнение**

In [4]:
import pandas as pd
import numpy as np

In [5]:
data = pd.read_excel('D:/Kuznetsov/Я_Проекты/Тестовое_М_Видео/Задание для Python.xlsx', sheet_name='Data' )

In [6]:
data_b = pd.read_excel('D:/Kuznetsov/Я_Проекты/Тестовое_М_Видео/Задание для Python.xlsx',sheet_name='Данные от бизнеса' )

In [7]:
data

,Объект хранения,SKU,Количество,Город
0,C111,SKU379,1,"Склады обслуживают все города, поэтому этот фи..."
1,C111,SKU15,9,"Склады обслуживают все города, поэтому этот фи..."
2,C111,SKU817,4,"Склады обслуживают все города, поэтому этот фи..."
3,C111,SKU351,8,"Склады обслуживают все города, поэтому этот фи..."
4,C111,SKU325,4,"Склады обслуживают все города, поэтому этот фи..."
...,...,...,...,...
803,ZZZ,SKU906,7,Уездный город X
804,ZZZ,SKU632,9,Уездный город X
805,ZZZ,SKU42,9,Уездный город X
806,ZZZ,SKU858,12,Уездный город X


In [8]:
data_b

,Город,Цепочка резерва
0,Уездный город N,C111-A234-S555-M666-Остальные объекты
1,Уездный город X,C111-A888-S777-M999-Остальные объекты


In [9]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 808 entries, 0 to 807
Data columns (total 4 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   Объект хранения  808 non-null    object
 1   SKU              808 non-null    object
 2   Количество       808 non-null    int64 
 3   Город            808 non-null    object
dtypes: int64(1), object(3)
memory usage: 25.4+ KB


*Проверим на пропуски*

In [10]:
data.isna().sum()

Объект хранения    0
SKU                0
Количество         0
Город              0
dtype: int64

In [11]:
data['Город'].value_counts()

Склады обслуживают все города, поэтому этот фильтр не нужен    456
Уездный город N                                                177
Уездный город X                                                175
Name: Город, dtype: int64

*Переименуем для удобства*

In [12]:
data.loc[(data['Город'] == 'Склады обслуживают все города, поэтому этот фильтр не нужен'),'Город'] = 'все города'

In [13]:
data['Город'].value_counts()

все города         456
Уездный город N    177
Уездный город X    175
Name: Город, dtype: int64

In [14]:
data_b

,Город,Цепочка резерва
0,Уездный город N,C111-A234-S555-M666-Остальные объекты
1,Уездный город X,C111-A888-S777-M999-Остальные объекты


**Разобьем цепочку резерка на разные столбцы**

In [15]:
new_df = data_b['Цепочка резерва'].str.split('-',expand=True)

In [16]:
new_df

,0,1,2,3,4
0,C111,A234,S555,M666,Остальные объекты
1,C111,A888,S777,M999,Остальные объекты


In [17]:
data_b = pd.concat([data_b,new_df],axis=1)

In [18]:
data_b

,Город,Цепочка резерва,0,1,2,3,4
0,Уездный город N,C111-A234-S555-M666-Остальные объекты,C111,A234,S555,M666,Остальные объекты
1,Уездный город X,C111-A888-S777-M999-Остальные объекты,C111,A888,S777,M999,Остальные объекты


In [19]:
df = data_b.drop('Цепочка резерва', 1)

In [20]:
df

,Город,0,1,2,3,4
0,Уездный город N,C111,A234,S555,M666,Остальные объекты
1,Уездный город X,C111,A888,S777,M999,Остальные объекты


**Создадим словарь Город - объект**

In [21]:
cities = df.set_index('Город').T.to_dict('list')

In [22]:
cities

{'Уездный город N': ['C111', 'A234', 'S555', 'M666', 'Остальные объекты'],
 'Уездный город X': ['C111', 'A888', 'S777', 'M999', 'Остальные объекты']}

In [23]:
#df.columns = ['Город','first','one','two','three', 'four']

**Создадим заготовку таблицы с необходимым порядком**

In [24]:
df1 = df.melt(id_vars = ['Город'])
df1

,Город,variable,value
0,Уездный город N,0,C111
1,Уездный город X,0,C111
2,Уездный город N,1,A234
3,Уездный город X,1,A888
4,Уездный город N,2,S555
5,Уездный город X,2,S777
6,Уездный город N,3,M666
7,Уездный город X,3,M999
8,Уездный город N,4,Остальные объекты
9,Уездный город X,4,Остальные объекты


In [25]:
df1 = df1.sort_values(by=['Город','variable']).reset_index().drop('index', 1)
df1

,Город,variable,value
0,Уездный город N,0,C111
1,Уездный город N,1,A234
2,Уездный город N,2,S555
3,Уездный город N,3,M666
4,Уездный город N,4,Остальные объекты
5,Уездный город X,0,C111
6,Уездный город X,1,A888
7,Уездный город X,2,S777
8,Уездный город X,3,M999
9,Уездный город X,4,Остальные объекты


In [26]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10 entries, 0 to 9
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   Город     10 non-null     object
 1   variable  10 non-null     object
 2   value     10 non-null     object
dtypes: object(3)
memory usage: 368.0+ bytes


In [27]:
data1 = data.copy()

**Сгруппируем**

In [28]:
data1 = data1.groupby(['Город','Объект хранения']).agg({'SKU':'nunique'}).reset_index()

In [29]:
data1

,Город,Объект хранения,SKU
0,Уездный город N,A234,60
1,Уездный город N,AAA,11
2,Уездный город N,JJJ,11
3,Уездный город N,LLL,11
4,Уездный город N,M666,11
5,Уездный город N,PFFF,11
6,Уездный город N,S555,62
7,Уездный город X,A888,60
8,Уездный город X,KKK,22
9,Уездный город X,M999,11


**Заменим строчку `все города` на название городов**

In [30]:
res = df1.merge(data1, left_on=['value'], right_on=['Объект хранения'], how='outer', indicator=True)
res

,Город_x,variable,value,Город_y,Объект хранения,SKU,_merge
0,Уездный город N,0,C111,все города,C111,456.0,both
1,Уездный город X,0,C111,все города,C111,456.0,both
2,Уездный город N,1,A234,Уездный город N,A234,60.0,both
3,Уездный город N,2,S555,Уездный город N,S555,62.0,both
4,Уездный город N,3,M666,Уездный город N,M666,11.0,both
5,Уездный город N,4,Остальные объекты,NaN,NaN,NaN,left_only
6,Уездный город X,4,Остальные объекты,NaN,NaN,NaN,left_only
7,Уездный город X,1,A888,Уездный город X,A888,60.0,both
8,Уездный город X,2,S777,Уездный город X,S777,60.0,both
9,Уездный город X,3,M999,Уездный город X,M999,11.0,both


In [31]:
all = res.query('_merge == "both" | _merge == "right_only"').reset_index(drop=True)

In [32]:
all

,Город_x,variable,value,Город_y,Объект хранения,SKU,_merge
0,Уездный город N,0,C111,все города,C111,456.0,both
1,Уездный город X,0,C111,все города,C111,456.0,both
2,Уездный город N,1,A234,Уездный город N,A234,60.0,both
3,Уездный город N,2,S555,Уездный город N,S555,62.0,both
4,Уездный город N,3,M666,Уездный город N,M666,11.0,both
5,Уездный город X,1,A888,Уездный город X,A888,60.0,both
6,Уездный город X,2,S777,Уездный город X,S777,60.0,both
7,Уездный город X,3,M999,Уездный город X,M999,11.0,both
8,NaN,NaN,NaN,Уездный город N,AAA,11.0,right_only
9,NaN,NaN,NaN,Уездный город N,JJJ,11.0,right_only


In [33]:
all.loc[(all['Город_y']=='все города'), 'Город_y'] = all['Город_x']

In [34]:
all

,Город_x,variable,value,Город_y,Объект хранения,SKU,_merge
0,Уездный город N,0,C111,Уездный город N,C111,456.0,both
1,Уездный город X,0,C111,Уездный город X,C111,456.0,both
2,Уездный город N,1,A234,Уездный город N,A234,60.0,both
3,Уездный город N,2,S555,Уездный город N,S555,62.0,both
4,Уездный город N,3,M666,Уездный город N,M666,11.0,both
5,Уездный город X,1,A888,Уездный город X,A888,60.0,both
6,Уездный город X,2,S777,Уездный город X,S777,60.0,both
7,Уездный город X,3,M999,Уездный город X,M999,11.0,both
8,NaN,NaN,NaN,Уездный город N,AAA,11.0,right_only
9,NaN,NaN,NaN,Уездный город N,JJJ,11.0,right_only


In [35]:
all = all.drop(['Город_x','variable','value','_merge'], 1)

**Получили необходимую таблицу**

In [36]:
all

,Город_y,Объект хранения,SKU
0,Уездный город N,C111,456.0
1,Уездный город X,C111,456.0
2,Уездный город N,A234,60.0
3,Уездный город N,S555,62.0
4,Уездный город N,M666,11.0
5,Уездный город X,A888,60.0
6,Уездный город X,S777,60.0
7,Уездный город X,M999,11.0
8,Уездный город N,AAA,11.0
9,Уездный город N,JJJ,11.0


**Словарь**

In [37]:
cities

{'Уездный город N': ['C111', 'A234', 'S555', 'M666', 'Остальные объекты'],
 'Уездный город X': ['C111', 'A888', 'S777', 'M999', 'Остальные объекты']}

**Теперь заменим объекты, которые не входят в цепочку резерва на `Остальные объекты` в соответствии со словарем `cities`**

In [38]:
def Other(row):
    for k, v in cities.items():
        if row['Город_y']==k:
            for elem in v:
                if row['Объект хранения']==elem:
                    return(elem)
            else:
                return 'Остальные объекты'

In [39]:
all['object'] = all.apply(Other, axis=1)

In [40]:
all

,Город_y,Объект хранения,SKU,object
0,Уездный город N,C111,456.0,C111
1,Уездный город X,C111,456.0,C111
2,Уездный город N,A234,60.0,A234
3,Уездный город N,S555,62.0,S555
4,Уездный город N,M666,11.0,M666
5,Уездный город X,A888,60.0,A888
6,Уездный город X,S777,60.0,S777
7,Уездный город X,M999,11.0,M999
8,Уездный город N,AAA,11.0,Остальные объекты
9,Уездный город N,JJJ,11.0,Остальные объекты


**Сгруппируем `Остальные объекты`**

In [41]:
all = all.groupby(['Город_y','object']).agg({'SKU': 'sum'}).reset_index(drop=False)

In [42]:
all

,Город_y,object,SKU
0,Уездный город N,A234,60.0
1,Уездный город N,C111,456.0
2,Уездный город N,M666,11.0
3,Уездный город N,S555,62.0
4,Уездный город N,Остальные объекты,44.0
5,Уездный город X,A888,60.0
6,Уездный город X,C111,456.0
7,Уездный город X,M999,11.0
8,Уездный город X,S777,60.0
9,Уездный город X,Остальные объекты,44.0


In [43]:
df1

,Город,variable,value
0,Уездный город N,0,C111
1,Уездный город N,1,A234
2,Уездный город N,2,S555
3,Уездный город N,3,M666
4,Уездный город N,4,Остальные объекты
5,Уездный город X,0,C111
6,Уездный город X,1,A888
7,Уездный город X,2,S777
8,Уездный город X,3,M999
9,Уездный город X,4,Остальные объекты


**Соединим для правильного порядка цепочки резерва**

In [44]:
Output = df1.merge(all, left_on=['Город', 'value'], right_on=['Город_y', 'object'], indicator=True)

In [45]:
Output

,Город,variable,value,Город_y,object,SKU,_merge
0,Уездный город N,0,C111,Уездный город N,C111,456.0,both
1,Уездный город N,1,A234,Уездный город N,A234,60.0,both
2,Уездный город N,2,S555,Уездный город N,S555,62.0,both
3,Уездный город N,3,M666,Уездный город N,M666,11.0,both
4,Уездный город N,4,Остальные объекты,Уездный город N,Остальные объекты,44.0,both
5,Уездный город X,0,C111,Уездный город X,C111,456.0,both
6,Уездный город X,1,A888,Уездный город X,A888,60.0,both
7,Уездный город X,2,S777,Уездный город X,S777,60.0,both
8,Уездный город X,3,M999,Уездный город X,M999,11.0,both
9,Уездный город X,4,Остальные объекты,Уездный город X,Остальные объекты,44.0,both


In [46]:
Output = Output.drop(['variable','Город_y','object','_merge'], 1)

### Получили `Output`

In [47]:
Output

,Город,value,SKU
0,Уездный город N,C111,456.0
1,Уездный город N,A234,60.0
2,Уездный город N,S555,62.0
3,Уездный город N,M666,11.0
4,Уездный город N,Остальные объекты,44.0
5,Уездный город X,C111,456.0
6,Уездный город X,A888,60.0
7,Уездный город X,S777,60.0
8,Уездный город X,M999,11.0
9,Уездный город X,Остальные объекты,44.0
